# Parareal Iterations

This notebook explains how to use the Parareal scheme in combination with the end-to-end model of the master's thesis “Fast, Accurate, and Scalable Numerical Wave Propagation: Enhancement by Deep Learning” by Luis Kaiser, supervised by Prof. Tsai (University of Texas Austin) and Prof. Klingenberg (University of Wuerzburg), in practice.

First, make sure that you have install all necessary libraries specified in `requirements.txt` using `pip` or `pip3` depending on your setup by running the command below.

In [ ]:
!pip3 install --upgrade pip
!pip3 install -r requirements.txt

We make this notebook easier to read with less complex dependencies in the code compared to the original implementation. The main difference is that this notebook is used to evaluate the performance of the trained end-to-end model, while in the other approach, we train the model during the Parareal iterations. This has the key benefit that the model is able to better leverage the Parareal scheme.

For this, we first input the velocity profile and an initial condition $u$ and $u_t$. Next, we take a first guess by applying the pre-trained end-to-end model multiple times (defined by variable `n_it`) to advance the wave front. We then use this information to enhance our solution according to the Parareal scheme. The thesis contains detailed information about the algorithm.
The coarse solver $G_{\Delta t^\star}$ and the fine solver $F_{\Delta t^\star}$ solve the wave equation numerically for a time step $\Delta t^\star$. We further define $\mathfrak{u}_{n+1} := F_{\Delta t^\star} \mathfrak{u}_n$ with $\mathfrak{u}_{n+1} := (u, u_t)$.

The Parareal scheme follows as:
\begin{equation}
    \mathfrak{u}_{n+1}^{k+1} = \mathcal{I} G_{\Delta t^\star} \mathcal{R} \mathfrak{u}_n^{k+1} + [F_{\Delta t^\star} \mathfrak{u}_n^k - \mathcal{I} G_{\Delta t^\star} \mathcal{R} \mathfrak{u}_n^k], k = 0,1,...
\end{equation}
with the starting condition
\begin{equation}
    \mathfrak{u}_{n+1}^0 := \mathcal{I} G_{\Delta t^\star} \mathcal{R} \mathfrak{u}_n^0, n \in \mathbb{N}.
\end{equation}

In [ ]:
from parareal_iterations import get_model, pseudo_spectral_solutions, visualize_parareal
from config import get_params
from generate_data import one_iteration_pseudo_spectral_tensor, initial_condition_gaussian, get_velocity_model
from matplotlib import pyplot as plt
import sys
import torch

sys.path.append("..")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def get_parareal_solutions(
        vel_data_path = "data/crop_test.npz",
        res = 128,
        n_it = 10,
        n_parareal = 2,
        boundary_conditions = "periodic",
        model_res = 128,
        model_path = "results/saved_model_test.pt"
):
    param_dict = get_params()
    model = get_model(
        model_path = model_path,
        param_dict = param_dict,
    )
    model.eval()
    vel = get_velocity_model(vel_data_path)

    # computing initial condition using gaussian pulse (switch to pytorch tensor if needed)
    u_energy, u_elapse = initial_condition_gaussian(
        torch.from_numpy(vel),
        mode="energy_components",
        res_padded=res
    )

    ps_sol_tensor, ps_u_elapse = pseudo_spectral_solutions(u_energy, u_elapse, vel, n_it,
                                       param_dict["f_delta_x"],
                                       param_dict["f_delta_t"],
                                       param_dict["delta_t_star"])

    with torch.no_grad():
        parareal_tensor = parareal_scheme(
            model,
            torch.cat([u_energy, torch.from_numpy(vel).unsqueeze(dim=0).unsqueeze(dim=0)], dim=1),
            n_parareal,
            n_it,
            boundary_conditions,
            ps_u_elapse
        )


    print(f"Comparing results for {n_it} iterations of pseudo-spectral method (first row) vs "
          f"end-to-end model (second row), and Parareal scheme (last rows).")

    visualize_parareal(
        ps_sol_tensor,
        parareal_tensor,
        n_parareal,
        n_it,
        param_dict["f_delta_x"],
        vel,
        u_elapse
    )

    plt.show()



def parareal_scheme(
        model,
        u_0,
        n_parareal,
        n_snapshots,
        boundary_conditions,
        ps_u_elapse
):

    u_n = u_0.clone()
    vel = u_0[:,3].clone().unsqueeze(dim=0)
    batch_size, channel, width, height = u_n.shape
    if boundary_conditions == "absorbing":
        width, height = width * 2, height * 2

    # create tensor to save results during iterations; in "absorbing" boundary case,
    # we use padding because of pseudo-spectral method
    big_tensor = torch.zeros([n_parareal+1, n_snapshots, batch_size, channel - 1, width, height])

    # initial guess, first iteration without parareal
    print(f"Performing initial guess.")
    for n in range(n_snapshots-1):
        u_n1 = model(u_n, ps_u_elapse[n])
        big_tensor[0,n+1] = u_n1
        u_n = torch.cat((u_n1, vel), dim=1)

    # parareal iterations: k = 1, 2, ...
    for k in range(1,n_parareal+1):
        print(f"Performing Parareal iteration {k}.")

        big_tensor[k,0] = u_0[0,:3].clone()
        parareal_terms = get_parareal_terms(
            model, big_tensor[k].clone(),
            n_snapshots, vel.clone(), ps_u_elapse
        )
        new_big_tensor = torch.zeros([n_snapshots, batch_size, channel - 1, width, height])
        new_big_tensor[0] = u_0[:, :3].clone()

        for n in range(n_snapshots-1):
            u_n_k1 = torch.cat((new_big_tensor[n], vel), dim=1)
            u_n1_k1 = model(u_n_k1, ps_u_elapse[n]) + parareal_terms[n]
            new_big_tensor[n+1] = u_n1_k1

        if k < n_parareal:
            big_tensor[k+1] = new_big_tensor.clone()

    return big_tensor


def get_parareal_terms(
        model,
        big_pseudo_tensor,
        n_snapshots,
        vel,
        ps_u_elapse
):
    '''
    Parameters
    ----------
    model : (pytorch.Model) end-to-end model to advance a wave front
    big_pseudo_tensor : (pytorch tensor) tensor containing previous solution (high resolution due to pseudo-spectral cropping)
    n_snapshots : (int) number of iterations (number of iterations with length dt_star)
    vel : (pytorch tensor) velocity profile
    ps_u_elapse : (float) u physical components

    Returns
    -------
    get Parareal terms that can be computed in parallel
    '''

    with torch.no_grad():
        parareal_terms = torch.zeros(big_pseudo_tensor.shape)
        for s in range(n_snapshots):
            parareal_terms[s] = compute_parareal_term(model, torch.cat([big_pseudo_tensor[s], vel], dim=1), ps_u_elapse[s])
    return parareal_terms


def compute_parareal_term(
        model,
        u_n_k,
        u_elapse
):
    '''
    Parameters
    ----------
    model : (pytorch.Model) end-to-end model to advance a wave front
    u_n_k : (pytorch tensor) current wave field
    u_elapse: (float) u physical components

    Returns
    -------
    difference between Parareal terms of right-hand side of main Parareal equation (see thesis)
    '''

    res_fine_solver, _ = one_iteration_pseudo_spectral_tensor(u_n_k, u_elapse)
    res_model = model(u_n_k, u_elapse)

    return res_fine_solver.to(device) - res_model.to(device)

In [ ]:
get_parareal_solutions()